In [1]:
%pip install faiss-cpu
%pip install langchain langchain_core langchain_community openai sentence_transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
from openai import OpenAI

# Inicializa o cliente OpenAI com a URL base e a chave de API
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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", # Mude a chave de API para a sua chave OpenRouter
)

In [3]:
# Carregando Embeddings locais com Sentence Transformers

def load_csv_data():
    # Carrega os documentos do arquivo CSV
    # Certifique-se de que o arquivo 'knowledge_base.csv' está no mesmo diretório do script
    loader = CSVLoader(file_path = "base_de_conhecimento.csv")
    documents = loader.load()

    # Cria embeddings usando o modelo HuggingFace
    model_name = "all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # Cria o vetor de armazenamento FAISS a partir dos documentos e embeddings
    vectorstore = FAISS.from_documents(documents, embeddings)
    retriever = vectorstore.as_retriever()
    return retriever

# Carrega os dados do CSV e cria o retriever
retriever = load_csv_data()

C:\Users\davi2\AppData\Local\Temp\ipykernel_19308\3199623523.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\davi2\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Loop de interação com o usuário
print("Bem-vindo ao Oraculo TUPÃ_RAG!")
print("Digite 'sair' para encerrar a conversa.\n")

while True:
    # Solicita entrada do usuário
    user_input = input("Você: ")

    # Verifica se o usuário deseja sair
    if user_input.lower() == 'sair':
        print("Encerrando a conversa. Até logo!")
        break

    # Obtém documentos relevantes usando o retriever
    docs = retriever.get_relevant_documents(user_input)
    contexto = "\n".join([doc.page_content for doc in docs])

    # Cria o prompt para o modelo
    prompt = (
        "Você é um atendente de uma empresa. "
        "Seu trabalho é conversar com os clientes, consultando a base de conhecimentos da empresa, e dar uma resposta simples e precisa para ele, baseada na base de dados da empresa fornecida como contexto.\n\n"
        "Contexto:\n" + contexto # Inclui o contexto na mensagem de prompt
    )

    # Chamada API DeepSeek/OpenRouter
    response = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ],
        max_tokens=500 # Limite de tokens para a resposta (ajuste conforme necessário)
    )

    # Exibe a resposta do modelo
    if response:
        print("\nUsuário:", user_input)
        print("\nOraculo TUPÃ:", response.choices[0].message.content.strip())
    else:
        print("\n❌ Nenhuma resposta da API.")

Bem-vindo ao Oraculo TUPÃ_RAG!
Digite 'sair' para encerrar a conversa.



C:\Users\davi2\AppData\Local\Temp\ipykernel_19308\946943428.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_input)



Usuário: Me diga informações uteis

Oraculo TUPÃ: Aqui estão as principais informações úteis baseadas nos dados da empresa:

1. **Distribuição Regional**  
   - A região do **Guajará (Belém)** possui a maior concentração de unidades (9).  
   - Regiões como **Carajás** e **Tocantins** têm menos unidades, mas com variações significativas no volume de atendimentos.  

2. **Inconsistências**  
   - Algumas unidades apresentam **baixo cumprimento de metas** (ex.: Educação Física/Belém).  
   - Há unidades **sem registros de atendimento**, como Icoaraci (necessário verificar falha no registro).  

3. **Sazonalidade/Interrupção**  
   - **Nenhum atendimento registrado após junho** (julho a dezembro = 0), indicando possível interrupção ou falta de lançamento nos sistemas.  

4. **Atendimentos Realizados vs. Programados**  
   - **Unidades que superaram a meta**:  
     - Marabá (Carajás): **789 realizados** vs. 712 programados.  
     - CCBS (Belém/Guajará): **1.423 realizados** vs. 1.156 pr